In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import signal
import soundfile as sf
import keras
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.initializers import Constant
from keras.regularizers import l2
from keras.applications import ResNet101
from keras import backend as K
from keras.layers import PReLU
import cv2
import librosa

**Load the augmented audio data and target labels**

In [ ]:
X_train_1 = np.load('../input/spec-dataset-3/stft_dataset.npy')
Y_train_1 = np.load('../input/spec-dataset-3/target.npy')
X_train_2 = np.load('../input/spec-dataset-laug/stft_dataset.npy')
Y_train_2 = np.load('../input/spec-dataset-laug/target.npy')
X_train_3 = np.load('../input/spec-dataset-naug/stft_dataset.npy')
Y_train_3 = np.load('../input/spec-dataset-naug/target.npy')
X_train_4 = np.load('../input/spec-dataset-paug/stft_dataset.npy')
Y_train_4 = np.load('../input/spec-dataset-paug/target.npy')

X_train = np.concatenate((X_train_1, X_train_2, X_train_3, X_train_4))
Y_train = np.concatenate((Y_train_1, Y_train_2, Y_train_3, Y_train_4))
print(X_train.shape)
print(Y_train.shape)

**Shuffle the data**

In [ ]:
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)

X_train = X_train[indices]
Y_train = Y_train[indices]
print(X_train.shape)
print(Y_train.shape)

**Distribution of data**

In [ ]:
unique, counts = np.unique(Y_train, return_counts = True)
print(unique)
print(counts)

In [ ]:
no_classes = 4

**One hot encoding**

In [ ]:
Y_train = keras.utils.to_categorical(Y_train, num_classes = no_classes)
print(Y_train.shape)

**Normalize the data**

In [ ]:
X_train = librosa.util.normalize(X_train)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
input_shape = X_train.shape[-3:]
input_shape

**Train-test split**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.05, random_state=42, stratify = Y_train)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

**Train-val split**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify = y_train)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape, kernel_regularizer = l2(0.01)))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.01)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.01)))
model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.01)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(64, activation = 'relu')) 
model.add(Dense(32, activation = 'relu')) 
model.add(Dropout(0.3)) 
model.add(Dense(no_classes, activation = 'softmax'))

model.summary()

**Hyperparameters**

In [ ]:
batch_size = 256
no_epochs = 600
verbosity = 1
lr = 0.001

**Save the best model**

In [ ]:
import h5py

filepath="/kaggle/working/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callback = [checkpoint]

**Compile and train the model**

In [ ]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer = keras.optimizers.Adam(learning_rate = lr), metrics = ['accuracy'])
history = model.fit(X_train, y_train, batch_size = batch_size, epochs = no_epochs, verbose = verbosity, validation_data = (X_val, y_val), callbacks = [callback])

**Plot the loss and accuracies**

In [ ]:
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('training / validation accuracies')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="upper left")
plt.show()
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('training / validation loss values')
plt.ylabel('Loss value')
plt.xlabel('Epoch')
plt.legend(loc="upper left")
plt.show()

**Evaluate model performance on test set**

In [ ]:
best_model = tf.keras.models.load_model('./weights-improvement-228-0.91.hdf5') # replace with path of best model
prediction = best_model.predict(X_test)

pr = np.argmax(prediction, axis = 1)
gt = np.argmax(y_test, axis = 1)

percentage = np.sum((pr == gt).astype(int)) * 100 / pred.shape[0]
percentage

**Plot the confusion matrix**

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm =confusion_matrix(gt, pr)  
index = ['both wheeze and crackle absent','wheeze present crackle absent','crackle present wheeze absent', 'both wheeze and crackle present']  
columns = ['both wheeze and crackle absent','wheeze present crackle absent','crackle present wheeze absent', 'both wheeze and crackle present'] 
cm_df = pd.DataFrame(cm,columns,index)

for i in range(4):
    cm_df.iloc[i] = cm_df.iloc[i] / cm_df.iloc[i].sum()

plt.figure(figsize=(10,6))  
sns.heatmap(cm_df, annot=True)